In [ ]:
import numpy as np
import sys
from pathlib import Path

# Make `import src.*` work when running from nested directory
REPO_ROOT = Path.cwd().resolve()
if not (REPO_ROOT / "src").exists() and (REPO_ROOT.parent / "src").exists():
    REPO_ROOT = REPO_ROOT.parent.resolve()

if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))
    
from src import (
    ExperimentConfig,
    load_config,
    list_datasets,
    list_models,
    run_pipeline,
)

# Load a base YAML config from ./config/ (convenience wrapper)
cfg = load_config("base_no_pca.yaml")
#cfg.data.data_type=""

print("datasets:", list_datasets())
print("models:", list_models())

datasets: ['breast_cancer_tabular', 'cifar10', 'cifar100', 'fashion_mnist', 'geometrical-shapes', 'mnist', 'synthetic_shapes_2class', 'synthetic_shapes_3class', 'two_moons']
models: ['minicnn, minicnn_mnist', 'two_moons_mlp']


In [2]:
cfg = load_config("base_no_pca")   # fastest
cfg.data.root = "../data"          # notebooks/ -> ../data
cfg.data.download = False          # True if you need to download MNIST

res = run_pipeline(
    dataset_name="mnist",
    model_name="minicnn_mnist",
    cfg=cfg,
    max_points_for_scoring=200,    # keep small
)
print(res.eval.metrics)


KeyboardInterrupt: 